## Fixed Data:

In [1]:
pip install fredapi

In [2]:
import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

In [3]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO')
Risk_free_rate = Risk_free_rate.to_frame()
Risk_free_rate = Risk_free_rate.reset_index()
Risk_free_rate = Risk_free_rate.rename(columns={'index':'Date', 0:'RiskFree'})

## Get Multiple Attributes for Multiple Coins:

In [6]:
### Define a function to pull data according to input values(coin, feature)
### After running this method, it should return pulled data, and also store it in memory named as "coin_feature"

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature} data")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 'api_key': API_KEY})
    data_raw = pd.read_json(res.text, convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 't':'Date'})

    # Turn renamed dataframe into a global variable for future using
    name = f"{coin}_{feature}"
    globals()[name] = data

    # Return a dataframe
    return data

In [9]:
### Get all features(listed in endPoints) of all coins(listed in symbolListShort) 

from functools import reduce

for i in symbolListShort:
    print(f"Start Pulling {i}'s data:")

    # Preparation for merging
    frameName = f"{i}_Frames"
    locals()[frameName] = []

    # Get data
    for j in list(endPoints.keys()):
        locals()[frameName].append(PullData(i, j))

    # Merge to single dataframe
    print('Merging Dataframes...')
    dfName = f"{i}_All"
    locals()[dfName] = reduce(lambda  left,right: pd.merge(left,right, on='Date', how='inner'), locals()[frameName])
    locals()[dfName] = locals()[dfName].merge(Risk_free_rate)

    print(f"{i} data successfully pulled\n")
print('\nALL DATA SUCCESSFULLY PULLED!')

Start Pulling BTC's data:
Pulling BTC's Price data
Pulling BTC's MarketCap data
Pulling BTC's TransactionCount data
Pulling BTC's ActiveAddress data
Merging Dataframes
BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price data
Pulling ETH's MarketCap data
Pulling ETH's TransactionCount data
Pulling ETH's ActiveAddress data
Merging Dataframes
ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price data
Pulling LTC's MarketCap data
Pulling LTC's TransactionCount data
Pulling LTC's ActiveAddress data
Merging Dataframes
LTC data successfully pulled


ALL DATA SUCCESSFULLY PULLED!


In [8]:
LTC_All

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree
0,2013-05-20,NaN,5.610263e+07,6375,12574,0.01
1,2013-05-21,NaN,5.618734e+07,6248,12086,0.04
2,2013-05-22,3.060000,5.468296e+07,5319,10606,0.04
3,2013-05-23,3.050000,5.459866e+07,5939,11720,0.03
4,2013-05-24,3.050000,5.469870e+07,5997,11850,0.03
...,...,...,...,...,...,...
2322,2022-04-13,110.360163,7.732401e+09,131892,328691,0.26
2323,2022-04-14,107.430865,7.527953e+09,113646,302368,0.37
2324,2022-04-15,111.103397,7.786069e+09,121456,309623,NaN
2325,2022-04-18,111.231135,7.797417e+09,110081,289475,0.38
